In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<link rel="stylesheet" href="../../../common/dhds.css">
<div class="Table">
    <div class="Row">
        <div class="Cell grey left"> <img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_39_Features_Importances_modelos_clasificacion/Presentacion/img/M5_CLASE_39_portada.jpg" align="center" width="70%"/></div>
        <div class="Cell right">
            <div class="div-logo"><img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/common/logo_DH.png" align="center" width=70% /></div>
            <div class="div-curso">DATA SCIENCE</div>
            <div class="div-modulo">MÓDULO 5</div>
            <div class="div-contenido">Feature Importance en modelos de clasificación
</div>
        </div>
    </div>
</div>

### Agenda

---

- Calcular la importancia de los features en los árboles de decisión y en los modelos de ensamble.

- Interpretabilidad de los modelos. Library LIME.

### Introducción

---

En general nos concentramos en optimizar el rendimiento del modelo. 

Pero también es importante encontrar *las features del dataset* que contribuyen en el modelo, y cual es su relevancia.

Por ejemplo, si tenemos 1000 features para predecir la retención de los clientes.
- ¿Qué features son relevantes?
- ¿Cómo se pueden identificar?
- ¿Cómo medimos la importancia?

### Introducción

---

Los modelos parámetricos, que parten de una función de clasificación conocida, reducen el problema *a estimar los parámetros* que mejor ajusten al dataset.

Por ejemplo, en las *regresiones logísticas* **cada parámetro está asociado a una feature determinada**. 

Si las features están estandarizadas, podemos interpretar el tamaño de cada parámetro como **indicador de la importancia relativa** del feature asociado.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_39_Features_Importances_modelos_clasificacion/Presentacion/img/M5_CLASE_39_005_logistica.JPG" alt="logistica" width=30% height=25% />
---

Pero, los modelos basados en árboles de clasificación *no son paramétricos*; no tenemos coeficientes para ajustar.

Veamos como encontrar la importancia de las features para estos modelos.

<div class="div-dhds-fondo-1"> Importancia de las features en árboles CART
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Introducción

---

Primero presentamos el dataset y creamos un modelo CART.

Luego analizamos las features del dataset para determinar la importancia de cada una.

###  Dataset

---
Vamos a trabajar con datos usados para la predicción de divorcios. 

Las **observaciones** son *personas* a las cuales se les hace preguntas. Las **features** son las *respuestas a cada una de las 54 preguntas*.

La **clase** representa si la persona está en pareja (0) o divorciada (1).

Para consultar detalles del dataset y las 54 preguntas ver <a href="http://archive.ics.uci.edu/ml/datasets/Divorce+Predictors+data+set#">aquí</a>.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_39_Features_Importances_modelos_clasificacion/Presentacion/img/M5_CLASE_39_001_luismi.JPG" alt="luismi" width=60% height=45% />

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

Las features Atr1 hasta Atr54 representan *afirmaciones*. Por ejemplo, "yo conozco a mi cónyuge muy bien".

Las personas responden con  un valor entre 0 y 4, donde **0 -** indica que concuerda totalmente con la afirmación, y **4 -** que no coincide en absoluto. Entre 1 y 3 son valores intermedios.

El *atributo class* indica el estado civil de la persona. 0 - casado, 1 - divorciado.

In [ ]:
df = pd.read_csv('../Data/divorce.csv', sep=';')
print('Filas:', df.shape[0], 'Columnas:', df.shape[1])

In [ ]:
df.head(3)

Creamos la matriz de features y la variable target.

In [ ]:
X = df.drop(['Class'], axis=1)
y = df['Class']

La proporción de casos para cada clase es similar: *0 - casado (sin pensar en separarse), 1 - divorciado*.

In [ ]:
y.value_counts(normalize=True)

Tambien se pueden ver la cantidad de respuestas *por afirmación y por valor*.

In [ ]:
df.apply(pd.value_counts)

###  Arbol de decisión CART

---

Generamos el modelo de árboles de decisión CART con <a href="https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart">scikit-learn</a>.

Dejamos de lado el análisis de la performance. Por eso no dividimos en train y test.

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(criterion='gini',max_depth=4,min_samples_leaf=2)
dt.fit(X, y)

plt.figure(figsize=(12,5))  # set plot size (denoted in inches)
tree.plot_tree(dt,feature_names=df.columns[:-1],filled=True,rounded=True, fontsize=12)
plt.show()

El modelo nos indica que:

- selecciona la afirmación 18, "My spouse and I have similar ideas about how marriage should be" (Mi cónyuge y yo tenemos ideas similares sobre cómo debería ser el matrimonio)

   - Si no coincide con esta afirmación (*Respuesta entre 2 y 4*) , lo clasifica como *divorciado*.
   
   - Si coincide (*Respuesta entre 0 y 1*) , selecciona la 26, " I know my spouse's basic anxieties" (Conozco las ansiedades básicas de mi cónyuge.)
   
       - Si no coincide, lo clasifica como *divorciado*.
       
       - Si coincide, selecciona la afirmación 3, "When we need it, we can take our discussions with my spouse from the beginning and correct it" (Cuando lo necesitamos, podemos tomar nuestras discusiones con mi cónyuge desde el principio y corregirlo.)
       
            - Hasta una respuesta entre 0 y 3, lo clasifica como *casado*.
           
            - Solo si responde con 4 lo clasifica como *divorciado*.     
           
*No está mal el modelo!* ;-)         

###  Arbol de decisión CART

---
Repasemos los valores que vemos en cada nodo. 

In [ ]:
plt.figure(figsize=(12,1))
tree.plot_tree(dt,feature_names=df.columns[:-1],filled=True,rounded=True, impurity=True,
                fontsize=12, max_depth=0, class_names=True)
plt.show()

- el criterio de partición. *Atr18 <= 1.5*.
- el criterio de impureza. *gini = 0.5*.
- total de casos asignados al nodo. *samples = 170*.
- casos por clase. *value*.
- clase seleccionada. *class*.

### Pureza

---

Sabemos que para particionar cada nodo, seleccionamos el atributo que logra nodos hijos lo **más homogéneos (puros) posibles**.

Definimos un nodo como *puro*, informalmente, si contiene predominantemente observaciones de una misma clase.

<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_39_Features_Importances_modelos_clasificacion/Presentacion/img/M5_CLASE_39_003_impureza.png" alt="impureza" width=60% height=45% />

### Indice de impureza Gini

---

Formalmente, usamos el **índice de impureza Gini**.  

$$G = \sum_{k=1}^K \hat{p}_{mk} . (1 - \hat{p}_{mk}) = 1 - \sum_{k=1}^K (\hat{p}_{mk})^2$$

donde $\hat{p}_{mk}$ representa la proporción de observaciones de entrenamiento en la m-ésima región que pertenecen a la k-ésima clase.

Un valor *cercano a cero* indica que contiene mayoritariamente observaciones de una misma clase. Un valor *de 0.5 indica máxima impureza*.

Por ejemplo, el nodo:
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_39_Features_Importances_modelos_clasificacion/Presentacion/img/M5_CLASE_39_004_nodo.JPG" alt="nodo" width=20% height=10% />

Tiene a $G = 0.065$, que indica un grado alto de pureza. Y se comprueba viendo que contiene mayoritariamente observaciones de la clase 0.

Recalculamos el $G = 0.5$ del nodo raíz, el cual indica máximo nivel de impureza.

$$G = \sum_{k=1}^K \hat{p}_{mk} . (1 - \hat{p}_{mk}) = 1 - \sum_{k=1}^K (\hat{p}_{mk})^2$$

In [ ]:
total_observaciones_clase = pd.Series(y).value_counts()
print("Total observaciones por clase"); print(total_observaciones_clase)
total_observaciones = sum(total_observaciones_clase)
print("Total observaciones",total_observaciones)
proporciones_clases = total_observaciones_clase / total_observaciones
print ("Proporciones Clases"); print (proporciones_clases)

In [ ]:
G = 1 - sum( proporciones_clases ** 2 ); print ("G: ", round(G,2))

### Importancia de las features

---
Para calcular la importancia de las features usamos la propiedad <a href="
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.feature_importances_">feature_importances_</a> de `DecisionTreeClassifier`.

Solo *tres features tienen alguna importancia*, pues son las únicas que fueron usados para generar las particiones.

In [ ]:
importancia_features = pd.DataFrame(dt.feature_importances_, index = X.columns, columns=['importancia'])
importancia_features_sort = importancia_features.sort_values('importancia', ascending=False)
importancia_features_sort[0:5]

### Importancia de las features

---
La importancia **compara la medida de impureza antes y después de la partición**. Se lo conoce también como **Importancia de Gini o ganancia**.

*A mayor ganancia, más importancia*, ya que si la medida de impureza del nodo padre **es mayor** a la suma de la impureza de los hijos, *mejoramos la pureza*.

Calcula la *reducción total normalizada* del criterio (Gini) por feature. 

$$\Delta = I(padre) - \sum_{j \in hijos} \frac{N_j}{N}. I(hijo_j) $$

Donde 

* $I$ es la medida de impureza, 
* $N_j$ es el número de registros en el nodo hijo $j$ y 
* $N$ es el número de registros en el nodo padre.



Recreamos los valores de `dt.feature_importances_`, observando el árbol de decisión.

In [ ]:
plt.figure(figsize=(10,5))
tree.plot_tree(dt,feature_names=df.columns[:-1],filled=True,rounded=True, fontsize=12)
plt.show()

Viendo el cálculo

$$\Delta = I(padre) - \sum_{j \in hijos} \frac{N_j}{N}. I(hijo_j) $$

In [ ]:
ganancia_gini_Atr18 = 1.000 * 0.5 - (89/170) * 0.065 - (81/170) * 0.0000
ganancia_gini_Atr26 = (89/170) * 0.065 - (87/170) * 0.023 - (2/170) * 0.0000
ganancia_gini_Atr3  = (87/170) * 0.023 - (85/170) * 0.0000 - (2/170) * 0.5

norm = ganancia_gini_Atr18 + ganancia_gini_Atr26 + ganancia_gini_Atr3

print ("Atr18:", round(ganancia_gini_Atr18 / norm,3))
print ("Atr26:", round(ganancia_gini_Atr26 / norm,3))
print ("Atr3:", round(ganancia_gini_Atr3 / norm,3))

### Entropia

---
Scikit-learn nos ofrece otro criterio para particionar, `criterion='entropy'`.

En teoría de la información se define **Entropía** como una forma de medir el grado de desorganización en un sistema. 

Mide qué tan parecidos son los elementos de un sistema:
- un *valor mínimo de 0* indica que son totalmente iguales.
- un *valor máximo de 1*, el mayor grado de desorden posible.

$$D = - \sum_{k = 1}^K{\hat{p}_{mk} . log(\hat{p}_{mk})}$$

donde $\hat{p}_{mk}$ representa la proporción de observaciones de entrenamiento en la m-ésima región que pertenecen a la k-ésima clase

Observemos que el nodo raíz tiene la *máxima entropía*, ya que la cantidad de observaciones de cada clase son similares.

No necesariamente ambos criterios generan igual modelo.

In [ ]:
from sklearn import tree
dt_entropy = tree.DecisionTreeClassifier(criterion='entropy',max_depth=4,min_samples_leaf=2)
dt_entropy.fit(X, y)

plt.figure(figsize=(8,5))
tree.plot_tree(dt_entropy,feature_names=df.columns[:-1],filled=True,rounded=True, fontsize=12)
plt.show()

### Importancia de las features - mejora del modelo

---
Conocer las features más importantes nos ayuda a generar un árbol más eficiente, en términos de cómputo.

Nos permite *reducir el conjunto de features* en uno menor que genera un modelo con bajo o nulo costo en la performance.

In [ ]:
dt = tree.DecisionTreeClassifier(criterion='gini',max_depth=4,min_samples_leaf=2)
dt.fit(X, y)
if_select = pd.DataFrame({'atributo':X.columns, 'importancia': dt.feature_importances_})
if_select.sort_values('importancia', ascending = False).iloc[0:4, :]

In [ ]:
features_select = if_select.atributo.values[if_select.importancia.values>0]
features_select

In [ ]:
dt_select = tree.DecisionTreeClassifier(criterion='gini',max_depth=4,min_samples_leaf=2)
dt_select.fit(X.loc[:, features_select], y)

plt.figure(figsize=(8,5))
tree.plot_tree(dt_select,feature_names=features_select,filled=True,rounded=True, fontsize=12)
plt.show()

<div class="div-dhds-fondo-1"> Importancia de las features en ensambles
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Modelos de ensamble

---

Vamos a generar un modelo de <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html">random forest</a> sobre el mismo dataset y calcular la importancia de las features.

La importancia de los features se calcula como **la media de la importancia de los features de los árboles base**.

Recordemos que Random Forest aplica la técnica de bagging pero en lugar de utilizar todas las variables independientes para cada modelo, aplica muestreos con reposición. 

En nuestro ejemplo, genera 10 datasets (`n_estimators=10`), donde cada uno es un muestreo de las features tomadas de a M sobre el total P (54) de features. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(class_weight='balanced', random_state=17, n_estimators=10)
rf.fit(X, y)

Por lo tanto, la cantidad de features con importancia mayor a cero *es mayor* al modelo CART, ya que ahora se generan más árboles que tienen distintas features. 

Se calcula también con la propiedad `feature_importances_`. Veamos los primeras cuatro features en importancia:

In [ ]:
rf_select = pd.DataFrame({'atributo':X.columns, 'importancia': rf.feature_importances_})
rf_select.sort_values('importancia', ascending = False).iloc[0:4, :]

### Modelos de ensamble

---
Vamos a comprobar que la importancia de las features en los ensambles es *el promedio* de su importancia en los árboles base.

Con la propiedad `estimators_` obtenemos los árboles base del ensamble.

In [ ]:
print('Total de árboles base: ',len(rf.estimators_))
print('Primer árbol base: ',rf.estimators_[0])

Cada árbol base tiene su matriz de importancia de features.

Como los atributos originales son 54, la matriz tiene 54 elementos, donde cada uno representa la importancia de una feature.

In [ ]:
len(rf.estimators_[0].feature_importances_)

Calculemos *la importancia del feature Atr18*, a partir de los árboles base.

Sabemos que los atributos del dataset se llaman `Atrx`, con $x=1..54$.

Por lo tanto, el *elemento 17* en las matrices de importancia de cada árbol base, es el valor para el *Atr18*.

Sumando el valor que tiene en todas las matrices de los árboles base, y dividiendo por la cantidad de árboles base, llegamos a la importancia de la feature para el ensamble.

In [ ]:
suma = sum([rf.estimators_[i].feature_importances_[17] for i in range(len(rf.estimators_))])
    
print('Importancia calculada de Atr18 en el ensamble: ',round(suma/10,6))

In [ ]:
print('Importancia de Atr18 en el ensamble, según feature_importances: ')
rf_select[rf_select.atributo=='Atr18']

<div class="div-dhds-fondo-1"> Interpretabilidad de los modelos
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

### Introducción

---
Muchas veces estamos interesados en la *interpretabilidad de los modelos* además de su performance.

A veces no podemos entender por qué algunas de nuestras predicciones son correctas mientras que otras veces no, ni podemos rastrear el camino de decisión de nuestro modelo.

**<a href="https://lime-ml.readthedocs.io/en/latest/lime.html">LIME</a>** (Local Interpretable Model-agnostic Explanations), es una biblioteca de Python que explica cómo decide un modelo de una manera comprensible, generando explicaciones a nivel local.

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lime.lime_tabular import LimeTabularExplainer

#%matplotlib inline

Seguimos usando el dataset de respuestas, pero limitado a *las primeras siete features* con mayor importancia en el modelo de Random Forest.

In [ ]:
if_select = pd.DataFrame({'atributo':X.columns, 'importancia': rf.feature_importances_})
features = if_select.sort_values('importancia', ascending = False).iloc[0:7, :]['atributo'].values.tolist()
features

In [ ]:
X_lime = X[features] 
X_lime.head(3)

Separamos los datos en train y test, y creamos un modelo de ensamble Random Forest. *Ya que necesitamos conocer las predicciones*.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_lime, y, test_size=0.3, random_state = 123)
rc_exp = RandomForestClassifier(n_estimators=50, random_state = 123)
rc_exp.fit(X_train, y_train)

### LimeTabularExplainer

---
Vamos a crear una instancia de <a href="https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=limetabularexplainer#lime.lime_tabular.LimeTabularExplainer">LimeTabularExplainer</a>, para después generar explicaciones sobre una predicción determinada usando este objeto.

In [ ]:
X_train_explainer = np.array(X_train)
explainer = LimeTabularExplainer(X_train_explainer, 
                                 mode = "classification",
                                 training_labels = y_train,
                                 feature_names = X_train.columns, 
                                 categorical_features  = list(range(7)),
                                 discretize_continuous=False)

Los parámetros del constructor de `LimeTabularExplainer` que vamos a usar son:
* training_data – <font color="green"> X_train transformado en un numpy 2d array.</font>
* mode – “classification” or “regression”, <font color="green">vamos a usar classification</font>
* training_labels – <font color="green">la variable target en train.</font>
* feature_names – <font color="green">nombre de las columnas del dataset.</font>
* categorical_features – <font color="green">Lista de los índices de las columnas categóricas.</font> Todas.
* discretize_continuous – <font color="green">si True, todas las features no categóricas se discretizan.</font>
* discretizer – <font color="green">Tipo de discretización. ‘quartile’, ‘decile’ or ‘entropy’.</font>

### Explainer

---
Usando el objeto `explainer` y el método `explain_instance` vamos a generar explicaciones sobre una predicción concreta.

Por ejemplo, sobre la fila (persona) de índice 47 del dataset de test.

`explain_instance` recibe como argumentos:

* data_row - <font color="blue">la fila que analizamos, de tipo 1d numpy array.</font>
* predict_fn – <font color="blue">clasificador que devuelve los valores de la predicción como probabilidades.</font> Para Clasificación es classifier.predict_proba(). Para Regresiones es regressor.predict()
* num_features – <font color="blue">Número de features presentes en la explicación</font>. Traemos todas.

Y devuelve una instancia de tipo `explanation`.

In [ ]:
i = 47
data_row = np.array(X_test.iloc[i]) # necesito que sea un np array:
explanation = explainer.explain_instance(data_row, rc_exp.predict_proba, num_features=len(data_row))

### Explanation

---
La clase **<a href="https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#module-lime.explanation">explanation</a>** provee métodos para analizar y visualizar el resultado.

`as_list` devuelve una lista (feature, peso) correspondiente a la predicción de la fila.

In [ ]:
explanation.as_list()

La columna de las features representa los valores que tiene la fila 47.

In [ ]:
X_test.iloc[47]

### Explanation

---
**<a href="https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=explanation#lime.explanation.Explanation.show_in_notebook">show_in_notebook</a>** muestra un gráfico resumiendo la información.

In [ ]:
explanation.show_in_notebook(show_table=True)

El gráfico tiene tres partes:

* El panel de la izquierda muestra *la probabilidad predicha* por el modelo para el registro de índice 47.

* El panel del medio muestra *las features por orden de importancia*. Las features que tienen color *naranja* son compatibles con la clase 1 y las *azules* con la clase 0. 

* El panel de la derecha muestra nuevamente las features y sus valores en el registro de índice 47.

`as_pyplot_figure` devuelve un gráfico de barras similar al panel central de `show_in_notebook`.

In [ ]:
explanation.as_pyplot_figure();

<div class="div-dhds-fondo-1"> Conclusiones
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M5/CLASE_34_CART/Presentacion/img/M5_CLASE_34_separador.png" align="center" />

</div>

## Conclusiones

---

* Vimos que el *índice de impureza Gini*, junto con la *Importancia de Gini* son medidas útiles para realizar el split de un nodo en nodos más puros.

* El algoritmo CART aplica el criterio *Gini* o el de *Entropía*.

* La importancia de las features es la *reducción total normalizada del criterio (Gini) por feature*. Es decir, cuanto gana en pureza al pasar del nodo padre a los nodos hijos.

* La importancia de las features en los ensambles se calcula como *la media de la importancia de las features de los árboles base*. 

* Conocer las features más importantes nos ayuda a reducir el conjunto de features que componen el modelo y así disminuir su costo computacional.

* Lime es una library de Python que explica como se generó la predicción para una observación determinada.

<div class="div-dhds-fondo-1"> Hands-on
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Ejercicio

----

A partir de un conjunto reducido del dataset usados para la predicción de divorcios:

- Generamos un modelo CART con criterio Gini y determinamos la importancia de las features.

- Generamos un modelo Random Forest con criterio Gini y 50 árboles base. También determinamos la importancia de las features.

- Comparamos las primeras 5 features en importancia. Son iguales?

### Dataset

----
Seguimos usando el dataset de respuestas, pero limitado a 10 features.

In [ ]:
features = ['Atr19', 'Atr20', 'Atr28', 'Atr3', 'Atr17', 'Atr54', 'Atr29', 'Atr39', 'Atr25', 'Atr8']

In [ ]:
X_ej = X[features]
y = df['Class'] 
X_ej.head(3)

### Solución

---

### Ejercicio

----

A partir de un conjunto reducido del dataset usados para la predicción de divorcios:

- Generamos un modelo CART con criterio Gini y determinamos la importancia de las features.

- Generamos un modelo Random Forest con criterio Gini y 50 árboles base. También determinamos la importancia de las features.

- Comparamos las primeras 5 features en importancia. Son iguales?

### Dataset

----
Seguimos usando el dataset de respuestas, pero limitado a 10 features.

In [ ]:
features = ['Atr19', 'Atr20', 'Atr28', 'Atr3', 'Atr17', 'Atr54', 'Atr29', 'Atr39', 'Atr25', 'Atr8']

In [ ]:
X_ej = X[features]
y = df['Class'] 
X_ej.head(3)

### Modelo CART

----

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from sklearn import tree
dt_ej = tree.DecisionTreeClassifier(criterion='gini',max_depth=4,min_samples_leaf=2)
dt_ej.fit(X_ej, y)

plt.figure(figsize=(12,5))
tree.plot_tree(dt_ej,feature_names=X_ej.columns[:-1],filled=True,rounded=True, fontsize=12)
plt.show()

### Importancia de las features - CART

---
Calculamos la importancia de las features.

In [ ]:
if_dt_ej = pd.DataFrame({'atributo':X_ej.columns, 'importancia': dt_ej.feature_importances_})
if_dt_ej_sort = if_dt_ej.sort_values('importancia', ascending=False)
if_dt_ej_sort[0:5]

### Modelo Random Forest

----
Generamos un ensamble con 50 árboles base.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_ej = RandomForestClassifier(class_weight='balanced', random_state=17, n_estimators=50)
rf_ej.fit(X_ej, y)

Calculamos también la importancia de las features.

In [ ]:
if_rf_ej = pd.DataFrame({'atributo':X_ej.columns, 'importancia': rf_ej.feature_importances_})
if_rf_ej.sort_values('importancia', ascending = False).iloc[0:5, :]

### Comparación

----
Verificamos que la primera feature es igual en ambos casos.

Los restantes son iguales pero en distinto orden.

In [ ]:
# 5 primeros features CART.
l_dt =  if_dt_ej_sort[0:5]['atributo'].values.tolist()

# 5 primeros features RF.
l_rf = if_rf_ej.sort_values('importancia', ascending = False).iloc[0:5, :]['atributo'].values.tolist()

In [ ]:
l_total = pd.DataFrame(
    {'CART': l_dt,
     'R.Forest': l_rf
    })

l_total

<div class="div-dhds-fondo-1"> Referencias y Material Adicional
<img src="https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_2021_img/master/M3/CLASE_21_Regresion_Lineal_Simple/Presentacion/img/M3_CLASE_21_separador.png" align="center" />

</div>

### Referencias y Material Adicional

---

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.feature_importances_" target="_blank">Scikit-learn: feature_importances</a>

<a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html" target="_blank">Scikit-learn: plot_tree</a>

<a href="https://towardsdatascience.com/the-mathematics-of-decision-trees-random-forest-and-feature-importance-in-scikit-learn-and-spark-f2861df67e3" target="_blank">The mathematics of decision trees and random-forest and feature importance in scikit learn</a>

<a href="https://medium.com/the-artificial-impostor/feature-importance-measures-for-tree-models-part-i-47f187c1a2c3#:~:text=It%20is%20sometimes%20called%20%E2%80%9Cgini,all%20trees%20of%20the%20ensemble." target="_blank">Feature Importance Measures for Tree Models — Part I</a>

<a href="https://becominghuman.ai/feature-importance-measures-for-tree-models-part-ii-20c9ff4329b" target="_blank">Feature Importance Measures for Tree Models — Part II</a>

<a href="https://lime-ml.readthedocs.io/en/latest/lime.html" target="_blank">Documentación LIME</a>